In [1]:
import os
import glob
import numpy as np
import pandas as pd
import tifffile
from mish import Mish as mish
from pathlib import Path
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Sequential

### **Recharger les données d'entraînement, de validation et test en fichier numpy**

In [2]:
# Recharger les données après avoir vidé la mémoire
train_X = np.load('Data/data_MLP/train_X.npy')
train_y = np.load('Data/data_MLP/train_y.npy')

valid_X = np.load('Data/data_MLP/valid_X.npy')
valid_y = np.load('Data/data_MLP/valid_y.npy')
valid_id = np.load('Data/data_MLP/valid_id.npy')

test_X = np.load('Data/data_MLP/test_X.npy')
test_id = np.load('Data/data_MLP/test_id.npy')
train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, valid_id.shape, test_X.shape, test_id.shape

((451962, 32),
 (451962,),
 (153469, 32),
 (153469,),
 (153469,),
 (207485, 32),
 (207485,))

### **Encoder les labels entre 0 et 4 de sorte à matcher les prédictions des réseaux de neurones**

In [3]:
encoder = LabelEncoder()
encoder.fit(train_y)
train_y_enc = encoder.transform(train_y)
valid_y_enc = encoder.transform(valid_y)
np.unique(train_y), np.unique(train_y_enc), np.unique(valid_y), np.unique(valid_y_enc)

(array([1, 2, 3, 4, 5], dtype=uint8),
 array([0, 1, 2, 3, 4]),
 array([1, 2, 3, 4, 5], dtype=uint8),
 array([0, 1, 2, 3, 4]))

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(32)))
model.add(tf.keras.layers.Dense(64, activation="relu"))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation="relu"))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(5, activation="softmax"))

model.summary()

optimizer = tf.keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])

# Définir un callback pour sauver les poids de votre modèle sur les meilleures époque c'est à dire les moments où il s'améliorera sur le jeu de validation
Path('my_model').mkdir(exist_ok=True, parents=True)
checkpointpath = os.path.join('my_model','model') # chemin où sauver le modèle
callbacks = [tf.keras.callbacks.ModelCheckpoint(
              checkpointpath,
              verbose=1, # niveau de log
              monitor='val_acc', # nom de la métrique à surveiller
              save_best_only=True, # sauver uniquement le meilleur modèle
              #shuffle=True,
              save_weights_only=True)] # sauver uniquement les poids

BATCH_SIZE = 256
EPOCHS = 25
model.fit (train_X, train_y_enc, validation_data=(valid_X,valid_y_enc), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks)
# loss: 0.1080 - acc: 0.9614 - val_loss: 0.4343 - val_acc: 0.8908 (100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 10,757
Trainable params: 10,757
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
1758/1766 [============================>.] - ETA: 0s - loss: 0.4818 - acc: 0.8455
Epoch 00001: val_acc im

In [19]:
# Restaurer les poids du modèle sur la meilleure époque d'entraînement
model.load_weights(checkpointpath)

In [20]:
# S'assurer que c'est bien le meilleur modèle sur les époques d'entraînement
model.evaluate(valid_X,valid_y_enc,batch_size=256)

600/600 [==============================] - 1s 840us/step - loss: 0.3141 - acc: 0.9017


[0.3140886425971985, 0.9016935229301453]

### **Prédiction des classes sur le jeu de validation et évaluation en aggrégeant au niveau objet**

In [21]:
# Récupérer les probabilités prédites sur le jeu de validation
valid_prob = model.predict(valid_X,batch_size=256)
valid_prob.shape

(153469, 5)

In [22]:
# Retourner la classe correspondant à la probabilité la plus haute
valid_pred = np.argmax(valid_prob,axis=1) # axe 1 car ceci concerne chaque ligne
valid_pred.shape

(153469,)

In [23]:
# Je réencode les prédictions entre 1 et 5
valid_pred_enc = encoder.inverse_transform(valid_pred)
np.unique(valid_pred_enc)

array([1, 2, 3, 4, 5], dtype=uint8)

In [24]:
# Aggrégation au niveau objet
out_pred = []
unique_id = np.unique(valid_id)
for ID in unique_id :
    # Récupérer les prédictions des pixels appartenant au même objet
    pred = valid_pred_enc[np.where(valid_id==ID)]
    y_true = valid_y[np.where(valid_id==ID)]
    # Prendre la valeur majoritaire des prédictions sur les pixels
    out_pred.append([ np.bincount(y_true).argmax(), np.bincount(pred).argmax()]) #(Vérité terrain,Prédiction majoritaire)
out_pred = np.vstack(out_pred)
out_pred.shape

(558, 2)

In [25]:
# F1 score au niveau objet
f1_score(out_pred[:,0],out_pred[:,1],average='weighted')

0.8675284522916036

### **Prédire sur le jeu test et Préparer une soumission**

In [26]:
# Récupérer les probabilités prédites sur le jeu test
test_prob = model.predict(test_X,batch_size=256)
test_prob.shape

(207485, 5)

In [27]:
# Retourner la classe correspondant à la probabilité la plus haute
test_pred = np.argmax(test_prob,axis=1) # axe 1 car ceci concerne chaque ligne
test_pred.shape

(207485,)

In [28]:
# Je réencode les prédictions entre 1 et 5
test_pred_enc = encoder.inverse_transform(test_pred)
np.unique(test_pred_enc)

array([1, 2, 3, 4, 5], dtype=uint8)

In [29]:
# Aggrégation au niveau objet
agg_pred = []
unique_id = np.unique(test_id)
for ID in unique_id :
    # Récupérer les prédictions des pixels appartenant au même objet
    pred = test_pred_enc[np.where(test_id==ID)]
    # Prendre la valeur majoritaire des prédictions sur les pixels
    agg_pred.append([ ID, np.bincount(pred).argmax()]) #(ID,Prédiction majoritaire)
agg_pred = np.vstack(agg_pred)
agg_pred.shape

(800, 2)

In [30]:
df = pd.DataFrame({'ID':agg_pred[:,0],'Class':agg_pred[:,1]})
df.to_csv('Output/Soumission_GoFocus_MLP.csv',index=False)
df.head(5)

,ID,Class
0,4,5
1,7,3
2,8,5
3,9,3
4,10,3


In [31]:
# F1 Score au niveau objet
df_test = pd.read_csv('Data/Test_id_Label.csv') # Ce fichier vous sera fourni le 12 Novembre
f1_score(df_test.Class,df.Class,average='weighted')

0.8485908690152663

In [ ]:
#test 1: 0.8168138256780346 #Dense(1, activation=tf.nn.sigmoid) #50 epochs
#test 2: 0.858408874794737 #commenter Dense(1, activation=tf.nn.sigmoid) #50 epochs
#test 3: 0.8531057408753407 #commenter un dropout #50 epochs
#test 4: 0.8505898715655911 #learning rate 0.01 #50 epochs
#test 5: 0.8358556090157446 #learning rate 0.01 et 2 dropour #50 epochs
#test 6: 0.6681578964596924 #sans shuffle drop(2)
#test 7: 0.839245722036305 #sans shuffle drop(32)
#test 8: 0.8508922745770693 #sans shuffle drop(128)
#test 9: 0.8378991833114442 #avec shuffle drop(128)
#test 10: 0.8025656496225863 #avec shuffle drop(128) et sigmoid
#test 11: 0.8477687744222915 #sans sigmoid learning rate 0.001 sans shuffle
#test 12: 0.8403317047002465 #avec shuffle
#test 13: 0.8489819756056037 #1 drop avec shuffle
#test 14: 0.8502387911808984 #3 couches dense 
#test 15: 0.8437704547495312 #sans shuffle 3 couche dense learning rate 0.01
#test 16: 0.8445219779416981 #sans shuffle 3 couche dense learning rate 0.001
#test 17: 0.8102395489221204 #sans shuffle 3 couche dense learning rate 0.001 avec fonction sigmoid
#test 18: 0.010465116279069769 #sans shuffle 3 couche dense learning rate 0.01 avec fonction sigmoid
#test 19: 0.8264949456297309 #sans shuffle 3 couche dense learning rate 0.01 
#test 20: 0.815481059283146
#test 21: 0.8336245042433846 
#test 22: 0.8477613639540746 
#test 23: 0.8535262044634899 #